In [52]:
import numpy as np
import cvxpy as cp
import random
import pandas as pd
import math

In [53]:
n = 10000    # number of bidders
m = 10       # type of resources

A = pd.read_csv("A.csv")
pi = pd.read_csv("pi.csv")
pi = np.array(pi)
pi = pi.T
A = np.array(A)
b = np.array([1000 for _ in range(m)])  


In [54]:
x = cp.Variable(n)
ee = np.ones(n)
objective = cp.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= ee]
prob = cp.Problem(objective, constraints)
prob.solve()
opv1 = prob.value
print("offline optimal value, ", opv1)

offline optimal value,  282270.0744458285


In [55]:
def u(s,w):
    value = 0
    for i in range(m):
        value += w/m * cp.log(s[i])
    return value
def learning(k=50):
    A1 = A[:m, :k]
    # pi = np.array(pi)
    pi1 = pi[:,:k]
    x = cp.Variable(k)
    s = cp.Variable(m)
    # print(pi1.shape)
    # print(x.shape)
    obj = cp.Maximize(pi1@x + u(s,1))
    cons = [A1@x + s <= k/n * b, x>=0, x<=1]
    prob = cp.Problem(obj, cons)
    # prob.solve(solver="ECOS")
    prob.solve(solver="SCS")
    opt_sol = x.value
    dual_price = prob.constraints[0].dual_value
    opt_value = prob.value 
    return opt_sol, dual_price, opt_value

def decision(dual_price,k=50):
    # Make the decisions based on the dual prices
    AT = A.T
    xp = []
    xk = []
    for i in range(n):
        cri = (AT[i].dot(dual_price))
        if(pi[0][i] > cri):
            xp.append(1)
        else:
            xp.append(0)  # xt(p) 10000
        if i < k:
            xk.append(0)  #50
    # Check whether the resource constriant is satisfied
    for t in range(k,n):
        flag = 1
        for j in range(m):
            temp1 = A[j,t]*xp[t]
            temp2 = b[j] - (A[j, :t]).dot(np.array(xk))
            if temp1 > temp2:
                flag = 0
                break
        if flag == 1:
            xk.append(xp[t])
        else:
            xk.append(0)
    return xk

**K = 50**

In [56]:
k = 50 
opt_sol, dual_price, opt_value = learning(k)
l_decision = decision(dual_price, k)
opv2 = (pi.dot(l_decision))[0]

ratio = opv2/opv1
print("when k = ", k)
print("dual price is", dual_price)
print("objective value is ", opv2)
print("the competetive ratio is ", ratio)

when k =  50
dual price is [44.08107686 34.73455245 14.28391645 49.16567588 25.33065137 26.23536667
  9.97934682 25.22723773 13.64142675 39.05960145]
objective value is  202844.59559326863
the competetive ratio is  0.7186188475399198


**K = 100**

In [57]:
k = 100
opt_sol, dual_price, opt_value = learning(k)
l_decision = decision(dual_price, k)
opv2 = (pi.dot(l_decision))[0]
ratio = opv2/opv1
print("when k = ", k)
print("dual price is", dual_price)
print("objective value is ", opv2)
print("the competetive ratio is ", ratio)

when k =  100
dual price is [44.0952587  35.03066016 13.98477837 49.09057023 25.0396567  26.01271964
 10.18851327 25.08728669 13.9836784  39.14983996]
objective value is  259779.69410064444
the competetive ratio is  0.9203231855543359


**K = 200**

In [58]:
k = 200
opt_sol, dual_price, opt_value = learning(k)
l_decision = decision(dual_price, k)
opv2 = (pi.dot(l_decision))[0]
ratio = opv2/opv1
print("when k = ", k)
print("dual price is", dual_price)
print("objective value is ", opv2)
print("the competetive ratio is ", ratio)

when k =  200
dual price is [44.08538016 35.00755035 14.04530684 49.09774847 24.96418303 26.03348051
 10.09854051 25.08253542 13.98054604 39.17291032]
objective value is  261796.93642631202
the competetive ratio is  0.9274696828570627
